In [14]:

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

import warnings
warnings.filterwarnings('ignore')


In [15]:
#  Loading storage data
def load_data():
    df = pd.read_csv("data2.csv")
    
    # Renaming columns
    column_mapping = {col: col.strip().lower().replace('&', 'and') for col in df.columns}
    return df.rename(columns=column_mapping)

In [16]:
#seperating the company name and number(year) from the column name "Company"
def split_company_name_and_year(df):
    df[['company', 'year']] = df['company'].str.extract(r'^(.*?)-(\d{4})$')
#extract year is after "-"


In [17]:
# Advanced preprocessing
def advanced_preprocessing(df):
    df = df[df['esg_score'].notna()].copy()
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    
    # Feature engineering of different year
    df['years_active'] = df.groupby('company')['year'].transform(lambda x: x - x.min() + 1)
    df['year_diff'] = df.groupby('company')['year'].transform(lambda x: x.diff().fillna(1))
    
    # Auto fill missing values
    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    for col in num_cols:
        df[col] = df.groupby('company')[col].transform(
            lambda x: x.fillna(x.mean() if x.mean() > 0 else 0)
        )
    
    return df

In [18]:
# preapre data
df = load_data()
split_company_name_and_year(df)
processed_df = advanced_preprocessing(df)

# features engineering
X = processed_df.drop(columns=['esg_score', 'company', 'year'])
y = processed_df['esg_score']
groups = processed_df['company']


In [35]:
# Define the model
models = {
    'RandomForest': make_pipeline(
        SimpleImputer(strategy='mean'),
        StandardScaler(),
        RandomForestRegressor(
            n_estimators=150,
            max_depth=4,
            min_samples_leaf=3,
            random_state=42
        )
    ),
    'SVR': make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    SVR(kernel='rbf', C=1.0, epsilon=0.1)),

    'GradientBoosting': make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42
    )),
     'XGBoost': make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    XGBRegressor(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        random_state=42
    )),


    'AdaBoost': make_pipeline(
        SimpleImputer(strategy='mean'),
        StandardScaler(),
        AdaBoostRegressor(
            estimator=DecisionTreeRegressor(max_depth=3),
            n_estimators=100,
            learning_rate=0.5,
            random_state=42
        )
    ),



    'MLP': make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    MLPRegressor(
        hidden_layer_sizes=(100,),
        activation='relu',
        solver='adam',
        max_iter=500,
        random_state=42
    )
),

'XGBoost1': make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    XGBRegressor(
        n_estimators=1000,
        max_depth=2,
        learning_rate=0.1,
        random_state=42
    )
),



}


In [36]:
# Cross-validation
gkf = GroupKFold(n_splits=3)
print("Model evaluation：")
for name, model in models.items():
    mae_scores = []
    for train_idx, test_idx in gkf.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        mae_scores.append(mean_absolute_error(y_test, preds))
    
    print(f"{name:15} | MAE: {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")

Model evaluation：
RandomForest    | MAE: 5.51 ± 0.88
SVR             | MAE: 8.09 ± 2.08
GradientBoosting | MAE: 4.76 ± 0.65
XGBoost         | MAE: 4.08 ± 0.84
AdaBoost        | MAE: 4.54 ± 1.26
MLP             | MAE: 2988015.89 ± 4216450.21
XGBoost1        | MAE: 3.87 ± 1.09


In [44]:
# analysis of feature importance
print("\nXGBoost Top 5：")
rf_model = models['XGBoost1'].steps[2][1]
importances = pd.Series(rf_model.feature_importances_, index=X.columns)
print(importances.sort_values(ascending=False).head(5))



XGBoost Top 5：
certification            0.960217
current_gender           0.017804
emission_intensities     0.007470
total_waste_generated    0.007294
absolute_emission        0.003113
dtype: float32


In [38]:
# Final model training 
final_model = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    XGBRegressor(
        n_estimators=1000,
        max_depth=2,
        learning_rate=0.1,
        random_state=42
    )
).fit(X, y)

In [40]:
# Prediction model
def predict_esg(model, current_data):
    # generate time feather (not keep company/year）
    current_data = current_data.copy()
    current_data['years_active'] = current_data['year'] - current_data['year'].min() + 1
    current_data['year_diff'] = 1
    
    required_features = X.columns.tolist()  
    missing_features = set(required_features) - set(current_data.columns)
    
    # Fill in missing features
    for feat in missing_features:
        current_data[feat] = 0
    
    return model.predict(current_data[required_features])


In [41]:
def standardize_columns(df):
    column_mapping = {col: col.strip().lower().replace('&', 'and') for col in df.columns}
    return df.rename(columns=column_mapping)


# Process latest data 
def load_latest_row():
    # read csv
    df = pd.read_csv("consolidated_esg_single_row.csv")
    df = standardize_columns(df)
    latest_row = df.iloc[-1].to_dict()
    
    # trun in DataFrame
    latest_df = pd.DataFrame([latest_row])
    
    # add year column if not exist
    if 'year' not in latest_df.columns:
        latest_df['year'] = 2024  # 添加默认年份
    
    return latest_df

# load ESG score
def load_esg_scores():
    esg_df = pd.read_csv("esg_scored_result.csv")
    last_row = esg_df.iloc[-1]
    esg_score_1 = last_row.iloc[-2]  
    esg_score_2 = last_row.iloc[-1]  
    return esg_score_1, esg_score_2

# process new data 
latest_data = load_latest_row()

# get ESG scores
esg_score_1, esg_score_2 = load_esg_scores()

#add nessasary columns
latest_data['esg_score'] = esg_score_1
latest_data['grade'] = esg_score_2

processed_latest = advanced_preprocessing(latest_data)


In [42]:
pred_score = predict_esg(final_model, processed_latest)
print(f"Predict value ：{pred_score[0]:.2f}")

Predict value ：44.00


In [43]:
def convert_grade(score):
    if pd.isna(score):
        return None
    if score < 30:
        return 'D'
    elif score < 40:
        return 'C-'
    elif score < 50:
        return 'C'
    elif score < 60:
        return 'C+'
    elif score < 67:
        return 'B-'
    elif score < 73:
        return 'B'
    elif score < 78:
        return 'B+'
    elif score < 83:
        return 'A-'
    elif score < 88:
        return 'A'
    else:
        return 'A+'
    

    # Convert the predicted score to a letter grade
predicted_grade = convert_grade(pred_score[0])
print(f"Predicted grade: {predicted_grade}")

Predicted grade: C
